<a href="https://colab.research.google.com/github/sundaybest3/NounSmart_pronunciation/blob/main/nounsmart_pronunciation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install pandas gradio gTTS SpeechRecognition numpy Scipy list

In [ ]:
import pandas as pd
import gradio as gr
from gtts import gTTS
import os
from difflib import SequenceMatcher
from tempfile import NamedTemporaryFile
import random
import speech_recognition as sr
import wave
import numpy as np
from scipy.io.wavfile import write

# Load and preprocess the CSV data
csv_url = "https://raw.githubusercontent.com/sundaybest3/NounSmart_pronunciation/refs/heads/main/nouns_CE_visang.csv"
data = pd.read_csv(csv_url)
data = data[(data['Countability'] != 'uncountable') & (data['regularity'] == 'regular')]  # Exclude uncountable and irregular nouns
plural_nouns = data[['Page', 'Plural']].dropna().reset_index(drop=True)

# Ensure static directory exists for storing audio files
os.makedirs("static", exist_ok=True)

# Generate dropdown options with word count
def generate_page_dropdown():
    pages = plural_nouns['Page'].unique()
    dropdown = [f"{page} ({len(plural_nouns[plural_nouns['Page'] == page])} words)" for page in pages]
    return dropdown

# Function to generate audio for a word
def generate_audio(word):
    file_path = f"static/{word.replace(' ', '_')}.mp3"
    if not os.path.exists(file_path):
        tts = gTTS(text=word, lang='en', slow=True)  # Enable slow playback
        tts.save(file_path)
    return file_path

# Function to calculate accuracy
def calculate_accuracy(user_input, correct_word):
    return round(SequenceMatcher(None, user_input.lower(), correct_word.lower()).ratio() * 100, 2)

# App logic functions
def greet_user(nickname):
    return f"Welcome, {nickname}! Select a page from the textbook you want to practice."

def get_practice_page(page):
    page_number = int(page.split(' ')[0])  # Extract the numeric page
    page_data = plural_nouns[plural_nouns['Page'] == page_number]
    return page_data.reset_index(drop=True)

def get_random_word(page):
    page_number = int(page.split(' ')[0])  # Extract the numeric page
    page_data = plural_nouns[plural_nouns['Page'] == page_number]
    if page_data.empty:
        return ""
    random_word = random.choice(page_data['Plural'].tolist())
    return random_word

def generate_word_audio(word):
    if not word:
        return None
    audio_path = generate_audio(f"{word}, {word}, {word}")
    return audio_path

def practice(word, user_audio):
    if user_audio is None:
        return "No audio detected. Please record your pronunciation and try again."

    correct_word = word
    temp_file = NamedTemporaryFile(delete=False, suffix=".wav")
    temp_file.close()

    try:
        # Extract audio data and write to a .wav file
        sample_rate, audio_array = user_audio  # Gradio's audio input format
        audio_array = (audio_array * 32767).astype(np.int16)  # Scale to int16 for .wav format
        write(temp_file.name, sample_rate, audio_array)

        # Process the user's audio input
        recognizer = sr.Recognizer()
        with sr.AudioFile(temp_file.name) as source:
            audio_data = recognizer.record(source)
            try:
                user_pronunciation = recognizer.recognize_google(audio_data).lower()
            except sr.UnknownValueError:
                return "Sorry, we couldn't understand your pronunciation. Please try again."
            except sr.RequestError:
                return "Speech recognition service is unavailable. Please try later."

        # Calculate accuracy
        accuracy = calculate_accuracy(user_pronunciation, correct_word)
        return f"Your accuracy: {accuracy}% (Recognized: {user_pronunciation})"
    except Exception as e:
        return f"An error occurred while processing the audio: {str(e)}"
    finally:
        os.remove(temp_file.name)  # Clean up the temporary file

# Gradio Interface
with gr.Blocks() as main_interface:
    with gr.Row():
        nickname = gr.Textbox(label="Enter your nickname")
        greet_button = gr.Button("Submit")
    greet_output = gr.Textbox(label="Greeting", interactive=False)

    page_number = gr.Dropdown(choices=generate_page_dropdown(), label="Select a Page")
    with gr.Row():
        get_word_button = gr.Button("Get a Random Word")
        random_word_output = gr.Textbox(label="Random Word to Practice", interactive=False)

    play_button = gr.Button("Play Word")
    audio_output = gr.Audio(label="Listen to the Word")

    user_audio_input = gr.Audio(label="Record Your Pronunciation")
    practice_button = gr.Button("Check Pronunciation")
    results_output = gr.Textbox(label="Results", interactive=False)

    greet_button.click(greet_user, inputs=[nickname], outputs=[greet_output])

    get_word_button.click(get_random_word, inputs=[page_number], outputs=[random_word_output])

    play_button.click(generate_word_audio, inputs=[random_word_output], outputs=[audio_output])

    practice_button.click(practice, inputs=[random_word_output, user_audio_input], outputs=[results_output])

if __name__ == "__main__":
    main_interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f6c120bea019b69f98.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
